In [1]:
import pandas as pd
from pathlib import Path

In [2]:
train = "dataset/train.csv.zip"
stores_csv = "dataset/stores.csv"

In [3]:
if Path(train).suffix == ".zip":
    import zipfile
    with zipfile.ZipFile(train, "r") as zip_ref:
        zip_ref.extractall("unzipped")
        train = "unzipped/train.csv"

# EDA

In [4]:
df = pd.read_csv(train)

In [5]:
print(df.shape,"\n")
print(df.info(),"\n")

(3000888, 6) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB
None 



In [6]:
print(df.isnull().sum())

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64


## Date Field Analysis

In [7]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df["date"].sample(10)

138761    2013-03-19
1372474   2015-02-12
1343941   2015-01-27
1447933   2015-03-26
2631667   2017-01-20
2032962   2016-02-18
385568    2013-08-05
1180126   2014-10-26
1570655   2015-06-03
2463218   2016-10-17
Name: date, dtype: datetime64[ns]

In [8]:
df["year"] = pd.DatetimeIndex(df["date"]).year
df["month"] = pd.DatetimeIndex(df["date"]).month
df["day"] = pd.DatetimeIndex(df["date"]).day

print(df.groupby("year").size(),"\n")
print(df.groupby("month").size(),"\n")
print(df.groupby("day").size(),"\n")

year
2013    648648
2014    648648
2015    648648
2016    650430
2017    404514
dtype: int64 

month
1     276210
2     251262
3     276210
4     267300
5     276210
6     267300
7     276210
8     247698
9     213840
10    220968
11    213840
12    213840
dtype: int64 

day
1     99792
2     99792
3     99792
4     99792
5     99792
6     99792
7     99792
8     99792
9     99792
10    99792
11    99792
12    99792
13    99792
14    99792
15    99792
16    98010
17    98010
18    98010
19    98010
20    98010
21    98010
22    98010
23    98010
24    98010
25    90882
26    98010
27    98010
28    98010
29    90882
30    89100
31    57024
dtype: int64 



## Store Analysis

In [9]:
stores = pd.read_csv(stores_csv)

In [10]:
print(stores.shape,"\n")
print(stores.info())

(54, 5) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
None


In [11]:
stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [12]:
print(stores.groupby("city").agg("size").sort_values(ascending=False),"\n")
print(stores.groupby("state").agg("size").sort_values(ascending=False))

city
Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Latacunga         2
Ambato            2
Machala           2
Manta             2
Cayambe           1
Babahoyo          1
El Carmen         1
Daule             1
Ibarra            1
Esmeraldas        1
Loja              1
Libertad          1
Guaranda          1
Playas            1
Quevedo           1
Puyo              1
Riobamba          1
Salinas           1
dtype: int64 

state
Pichincha                         19
Guayas                            11
Manabi                             3
Azuay                              3
Santo Domingo de los Tsachilas     3
Los Rios                           2
El Oro                             2
Cotopaxi                           2
Tungurahua                         2
Esmeraldas                         1
Bolivar                            1
Chimborazo                         1
Imbabura                           1
Loja                               1
Pastaza         

In [13]:
print(stores.groupby("type").agg("size").sort_values(ascending=False),"\n")
print(stores.groupby("cluster").agg("size").sort_values(ascending=False))

type
D    18
C    15
A     9
B     8
E     4
dtype: int64 

cluster
3     7
10    6
6     6
15    5
13    4
14    4
1     3
8     3
4     3
11    3
9     2
2     2
7     2
5     1
12    1
16    1
17    1
dtype: int64


In [14]:
train_store = pd.merge(df, stores, on="store_nbr", how="left")

In [15]:
train_store.sample(10)

,id,date,store_nbr,family,sales,onpromotion,year,month,day,city,state,type,cluster
2531537,2531537,2016-11-24,4,DAIRY,658.0,18,2016,11,24,Quito,Pichincha,D,9
424573,424573,2013-08-27,21,POULTRY,0.0,0,2013,8,27,Santo Domingo,Santo Domingo de los Tsachilas,B,6
1261063,1261063,2014-12-10,42,BABY CARE,0.0,0,2014,12,10,Cuenca,Azuay,D,2
513795,513795,2013-10-16,25,HOME CARE,0.0,0,2013,10,16,Salinas,Santa Elena,D,1
1843349,1843349,2015-11-03,30,BEAUTY,0.0,0,2015,11,3,Guayaquil,Guayas,C,3
2142138,2142138,2016-04-20,14,DELI,179.0,0,2016,4,20,Riobamba,Chimborazo,C,7
2277735,2277735,2016-07-05,19,DELI,336.0,4,2016,7,5,Guaranda,Bolivar,C,15
699735,699735,2014-01-29,42,BEVERAGES,0.0,0,2014,1,29,Cuenca,Azuay,D,2
847054,847054,2014-04-22,26,EGGS,26.0,0,2014,4,22,Guayaquil,Guayas,D,10
543979,543979,2013-11-02,22,CLEANING,0.0,0,2013,11,2,Puyo,Pastaza,C,7


In [16]:
with pd.option_context('display.float_format','{:.0f}'.format):
    gb = train_store.groupby("store_nbr").agg(total_sales=("sales", "sum"))
    gb = pd.merge(gb, stores, on="store_nbr", how="left")
    print(gb.sort_values(by="total_sales", ascending=False).to_string(index=False))

 store_nbr  total_sales          city                          state type  cluster
        44     62087553         Quito                      Pichincha    A        5
        45     54498010         Quito                      Pichincha    A       11
        47     50948310         Quito                      Pichincha    A       14
         3     50481910         Quito                      Pichincha    D        8
        49     43420096         Quito                      Pichincha    A       11
        46     41896062         Quito                      Pichincha    A       14
        48     35933130         Quito                      Pichincha    A       14
        51     32911490     Guayaquil                         Guayas    A       17
         8     30494287         Quito                      Pichincha    D        8
        50     28653021        Ambato                     Tungurahua    A       14
        11     28325558       Cayambe                      Pichincha    B        6
    

In [17]:
with pd.option_context('display.float_format','{:.0f}'.format):
    gb = gb.groupby("cluster") \
        .agg(
            cluster_sales=("total_sales", "sum"),
            cluster_size=("store_nbr", "count"),
            average_sales_per_store=("total_sales", "mean"),
            lowest_earner=("total_sales", "min"),
            highest_earner=("total_sales", "max"),
            store_std=("total_sales", "std"),
        ) \
        .sort_values(by="cluster_sales",ascending=False)
    print(gb)

         cluster_sales  cluster_size  average_sales_per_store  lowest_earner  \
cluster                                                                        
14           157430523             4                 39357631       28653021   
6            114254387             6                 19042398        9255506   
8            107928247             3                 35976082       26952050   
11           100614276             3                 33538092        2696170   
10            85324430             6                 14220738        7755122   
3             75628703             7                 10804100        5951796   
13            72102249             4                 18025562       11216167   
5             62087553             1                 62087553       62087553   
15            55296949             5                 11059390        9613906   
1             54376752             3                 18125584       11420998   
4             49443443             3    